In [1]:
from sqlalchemy import create_engine
from sqlalchemy.orm import Session

engine = create_engine("sqlite:///userdata.db")

In [2]:
from sqlalchemy.orm import DeclarativeBase

class Base(DeclarativeBase):
    pass

In [3]:
from sqlalchemy import String, DateTime, func
from sqlalchemy.orm import mapped_column

class Userdata(Base):
    __tablename__ = "userdata"
    user_id = mapped_column(String(32), primary_key=True, nullable=False)
    user_data = mapped_column(String(640))
    time_stamp = mapped_column(DateTime(timezone=True), server_default=func.now(), onupdate=func.now())

In [ ]:
from sqlalchemy import String, DateTime
from sqlalchemy.orm import mapped_column

class Userdata(Base):
    __tablename__ = "userdata"
    user_id = mapped_column(String(32), primary_key=True, nullable=False)
    user_data = mapped_column(String(640))
    time_stamp = mapped_column(DateTime(timezone=True))

In [ ]:
print(Base.metadata.tables)

In [ ]:
print(Userdata)

Database is created after the step below. See userdata.db file in folder

In [4]:
Base.metadata.create_all(engine)

In section below, need to manually commit and close session

In [5]:
from datetime import datetime

session = Session(engine)
user = Userdata(user_id="Brad", user_data="Data from brad")
session.add(user)
user = Userdata(user_id="Larry", user_data="More data")
session.add(user)
user = Userdata(user_id="Jane", user_data="Even more data")
session.add(user)
session.commit()
session.close()

In [13]:
from sqlalchemy import update

def db_update(session, id, data):
    stmt = (update(Userdata)
            .where(Userdata.user_id == id)
            .values(user_data=data))
    session.execute(stmt)

In [14]:
db_update(session, "Brad", "New New New Bradley")
user = Userdata(user_id="Sally", user_data="Sally data")
session.add(user)
session.commit()

In [ ]:
from datetime import datetime

session = Session(engine)
user = Userdata(user_id="Brad", user_data="Data from brad", time_stamp=datetime.now())
session.add(user)
user = Userdata(user_id="Larry", user_data="More data", time_stamp=datetime.now())
session.add(user)
user = Userdata(user_id="Jane", user_data="Even more data", time_stamp=datetime.now())
session.add(user)
session.commit()
session.close()

if you just use Session class, conext manager does not automatically write data to DB at end of block, or rollback if error. need to call *session.commit()* before end of block


In [ ]:
with Session(engine) as session:
    user = Userdata(user_id="Allan", user_data="Data from Allan", time_stamp=datetime.now())
    session.add(user)
    user = Userdata(user_id="Chad", user_data="More Chad data", time_stamp=datetime.now())
    session.add(user)
    user = Userdata(user_id="Mary", user_data="Even more Mary data", time_stamp=datetime.now())
    session.add(user)
    session.commit()


In the section below, the sessiomaker's *begin* method creates a context manager that will run purge() and commit() at the end of the block so they do not need to be explixitly called using *session.commit()*. Also takes care of rollback if write fails. See https://docs.sqlalchemy.org/en/14/orm/session_basics.html#getting-a-session for more ways to do this

In [ ]:
from sqlalchemy.orm import sessionmaker

Session2 = sessionmaker(engine)

with Session2.begin() as session:
    user1 = Userdata(user_id="xx", user_data="Data from Phil", time_stamp=datetime.now())
    user2 = Userdata(user_id="yy", user_data="More Karen data", time_stamp=datetime.now())
    user3 = Userdata(user_id="ss", user_data="Gary data", time_stamp=datetime.now())
    session.add_all([user1, user2, user3])

In [ ]:
from sqlalchemy import select

stmt = select(Userdata)
results = session.execute(stmt).scalars()
for x in results:
    print(x.user_id)

In [ ]:
user = Userdata(user_id="Flusher", user_data="Flushed?", time_stamp=datetime.now())
session.add(user)
stmt = select(Userdata).execution_options(autoflush=False)
results = session.execute(stmt).scalars()
for x in results:
    print(x.user_id)

In [ ]:
session.flush()

In [ ]:
stmt = select(Userdata).execution_options(autoflush=False)
results = session.execute(stmt).fetchall()
for x in results:
    print(type(x[0]))

In [ ]:
stmt = select(Userdata).execution_options(autoflush=False)
results = session.execute(stmt).scalars()
for x in results:
    print(x.user_data)

In [ ]:
stmt = select(Userdata.user_data, Userdata.time_stamp).where(Userdata.user_id=="Brad")
results = session.execute(stmt).one()
print(results)
print(results.time_stamp)

In [ ]:
session.close()